# Necessary packages and Installation

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import pandas as pd
import re
from bs4 import BeautifulSoup

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import spacy

2024-10-10 12:23:21.686331: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-10 12:23:21.687691: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-10 12:23:21.707482: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 12:23:21.707500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 12:23:21.708171: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

# Dataset of Stack Overflow

In [2]:
## Loading SO posts
so_questions = pd.read_csv('/home/kha060/PhD/Developer Challenges LLM/Stack_Overflow/LLMs-Related-SO-Questions.csv') ## You may need to change the path based on the necessity
df_selected_columns = so_questions[['Id', 'Title', 'Body', 'CreationDate']]

## Combining Title and Body

df_selected_columns['Text'] = df_selected_columns['Title'] + ' ' +df_selected_columns['Body']
df_selected_columns['OriginalText'] = df_selected_columns['Title'] + ' ' +df_selected_columns['Body']
# df_selected_columns['OriginalTitle'] = df_selected_columns['Title']

/tmp/ipykernel_2336646/1000779888.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['Text'] = df_selected_columns['Title'] + ' ' +df_selected_columns['Body']
/tmp/ipykernel_2336646/1000779888.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['OriginalText'] = df_selected_columns['Title'] + ' ' +df_selected_columns['Body']


In [3]:
df_selected_columns

,Id,Title,Body,CreationDate,Text,OriginalText
0,32333312,How to extract chunks from BIO chunked sentenc...,"<p>Give an input sentence, that has <a href=""h...",2015-09-01 13:45:07,How to extract chunks from BIO chunked sentenc...,How to extract chunks from BIO chunked sentenc...
1,33695244,Use Completion Suggester to match against all ...,<p>I'd like to know if it's possible to use <a...,2015-11-13 14:41:40,Use Completion Suggester to match against all ...,Use Completion Suggester to match against all ...
2,33941091,Chunking documents to test for plagiarism,<p>I am building a plagiarism checker for text...,2015-11-26 14:28:31,Chunking documents to test for plagiarism <p>I...,Chunking documents to test for plagiarism <p>I...
3,34090734,How to use nltk regex pattern to extract a spe...,<p>I have written the following regex to tag c...,2015-12-04 14:37:13,How to use nltk regex pattern to extract a spe...,How to use nltk regex pattern to extract a spe...
4,34318427,itextsharp: words are broken when splitting te...,<p>I want to highlight several keywords in a s...,2015-12-16 17:30:43,itextsharp: words are broken when splitting te...,itextsharp: words are broken when splitting te...
...,...,...,...,...,...,...
8588,78981951,How do I set the random seed for Chroma DB?,<p>I'm experimenting with different embedding ...,2024-09-13 11:27:10,How do I set the random seed for Chroma DB? <p...,How do I set the random seed for Chroma DB? <p...
8589,78982153,How to send a parameter directly to LLM in lan...,<p>Currently I can send grammar in Huggingface...,2024-09-13 12:33:25,How to send a parameter directly to LLM in lan...,How to send a parameter directly to LLM in lan...
8590,78984423,Azure Document Intelligence Custom Classificat...,<p>I am making 2 API calls using Python.</p>\n...,2024-09-14 06:15:24,Azure Document Intelligence Custom Classificat...,Azure Document Intelligence Custom Classificat...
8591,78984512,Protecting Routes in an Edge-Runtime with T3 S...,<p>How can I authorize users on my OpenAI rout...,2024-09-14 07:17:11,Protecting Routes in an Edge-Runtime with T3 S...,Protecting Routes in an Edge-Runtime with T3 S...


# Functions to Pre-Process the Data

In [4]:
## Stop Words
nltk.download('stopwords')
additional_stopwords=['[', ']','#', '`', '"', '@', '*']
#additional_stopwords=['[', ']','#', '`', '"', '@', '*']
stop_words = set(stopwords.words('english'))
stop_words.update(additional_stopwords)
#stop_words

## Remove stop words
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to /home/kha060/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df_selected_columns['Title'] = df_selected_columns['Title'].apply(remove_stopwords)
df_selected_columns['Body'] = df_selected_columns['Body'].apply(remove_stopwords)
df_selected_columns['Text'] = df_selected_columns['Text'].apply(remove_stopwords)

/tmp/ipykernel_2336646/4285654231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['Title'] = df_selected_columns['Title'].apply(remove_stopwords)
/tmp/ipykernel_2336646/4285654231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['Body'] = df_selected_columns['Body'].apply(remove_stopwords)
/tmp/ipykernel_2336646/4285654231.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [6]:
def remove_quotes(text):
    clean_text = re.sub(r"[\'\"]", '', text)
    return clean_text

In [7]:
df_selected_columns['Title'] = df_selected_columns['Title'].apply(remove_quotes)
df_selected_columns['Body'] = df_selected_columns['Body'].apply(remove_quotes)
df_selected_columns['Text'] = df_selected_columns['Text'].apply(remove_quotes)
df_selected_columns['OriginalText'] = df_selected_columns['OriginalText'].apply(remove_quotes)

/tmp/ipykernel_2336646/2762558621.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['Title'] = df_selected_columns['Title'].apply(remove_quotes)
/tmp/ipykernel_2336646/2762558621.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['Body'] = df_selected_columns['Body'].apply(remove_quotes)
/tmp/ipykernel_2336646/2762558621.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [8]:
## Remove link and its content
def remove_a_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    for a_tag in soup.find_all('a'):
        a_tag.extract()
    return str(soup)

In [9]:
df_selected_columns['Title'] = df_selected_columns['Title'].apply(remove_a_tags)
df_selected_columns['Body'] = df_selected_columns['Body'].apply(remove_a_tags)
df_selected_columns['Text'] = df_selected_columns['Text'].apply(remove_a_tags)
df_selected_columns['OriginalText'] = df_selected_columns['OriginalText'].apply(remove_a_tags)

/tmp/ipykernel_2336646/2119165633.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
/tmp/ipykernel_2336646/4223207516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['Title'] = df_selected_columns['Title'].apply(remove_a_tags)
/tmp/ipykernel_2336646/4223207516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_column

In [10]:
# Define a regular expression pattern to match code blocks
code_pattern = re.compile(r'<code>(.*?)</code>', re.DOTALL)

# Define a function to clean HTML tags and code blocks from text
def clean_html_and_code(text):
    soup = BeautifulSoup(text, "html.parser")
    # Remove code blocks
    for code_block in soup.find_all("code"):
        code_block.decompose()
    # Remove HTML tags
    cleaned_text = soup.get_text(separator=" ")
    return cleaned_text

In [11]:
df_selected_columns['Title'] = df_selected_columns['Title'].apply(clean_html_and_code)
df_selected_columns['Body'] = df_selected_columns['Body'].apply(clean_html_and_code)
df_selected_columns['Text'] = df_selected_columns['Text'].apply(clean_html_and_code)
df_selected_columns['OriginalText'] = df_selected_columns['OriginalText'].apply(clean_html_and_code)

/tmp/ipykernel_2336646/447692465.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/tmp/ipykernel_2336646/607180627.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['Title'] = df_selected_columns['Title'].apply(clean_html_and_code)
/tmp/ipykernel_2336646/607180627.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_col

In [12]:
df_selected_columns

,Id,Title,Body,CreationDate,Text,OriginalText
0,32333312,extract chunks BIO chunked sentences? - python,"Give input sentence, : [(What, B-NP), (is,...",2015-09-01 13:45:07,extract chunks BIO chunked sentences? - python...,How to extract chunks from BIO chunked sentenc...
1,33695244,Use Completion Suggester match ngrams query,Id like know possible use match ngrams query....,2015-11-13 14:41:40,Use Completion Suggester match ngrams query I...,Use Completion Suggester to match against all ...
2,33941091,Chunking documents test plagiarism,I building plagiarism checker text files. ever...,2015-11-26 14:28:31,Chunking documents test plagiarism I building...,Chunking documents to test for plagiarism I a...
3,34090734,use nltk regex pattern extract specific phrase...,I written following regex tag certain phrases ...,2015-12-04 14:37:13,use nltk regex pattern extract specific phrase...,How to use nltk regex pattern to extract a spe...
4,34318427,itextsharp: words broken splitting textchunk w...,I want highlight several keywords set PDF file...,2015-12-16 17:30:43,itextsharp: words broken splitting textchunk w...,itextsharp: words are broken when splitting te...
...,...,...,...,...,...,...
8588,78981951,set random seed Chroma DB?,Im experimenting different embedding models. C...,2024-09-13 11:27:10,set random seed Chroma DB? Im experimenting d...,How do I set the random seed for Chroma DB? I...
8589,78982153,send parameter directly LLM langchain,Currently send grammar HuggingfaceEndpoint par...,2024-09-13 12:33:25,send parameter directly LLM langchain Current...,How to send a parameter directly to LLM in lan...
8590,78984423,Azure Document Intelligence Custom Classificat...,I making 2 API calls using Python. url_1 w...,2024-09-14 06:15:24,Azure Document Intelligence Custom Classificat...,Azure Document Intelligence Custom Classificat...
8591,78984512,Protecting Routes Edge-Runtime T3 Stack / Verc...,How authorize users OpenAI route everyone call...,2024-09-14 07:17:11,Protecting Routes Edge-Runtime T3 Stack / Verc...,Protecting Routes in an Edge-Runtime with T3 S...


In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/kha060/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
nlp = spacy.load("en_core_web_sm")

In [15]:
## Stemming and Lemmatize
porter = PorterStemmer()

def stem_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [porter.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

In [16]:
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return ' '. join(lemmatized_tokens)


In [17]:
df_selected_columns['Title'] = df_selected_columns['Title'].apply(lemmatize_text)
df_selected_columns['Body'] = df_selected_columns['Body'].apply(lemmatize_text)
df_selected_columns['Text'] = df_selected_columns['Text'].apply(lemmatize_text)

/tmp/ipykernel_2336646/533237907.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['Title'] = df_selected_columns['Title'].apply(lemmatize_text)
/tmp/ipykernel_2336646/533237907.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['Body'] = df_selected_columns['Body'].apply(lemmatize_text)
/tmp/ipykernel_2336646/533237907.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

# Storing the Cleaned Data

In [18]:
columns_list = ['Id', 'Title', 'Text', 'OriginalText', 'CreationDate']
df_to_save = df_selected_columns[columns_list]
df_to_save

,Id,Title,Text,OriginalText,CreationDate
0,32333312,extract chunk BIO chunk sentence ? - python,extract chunk BIO chunk sentence ? - python ...,How to extract chunks from BIO chunked sentenc...,2015-09-01 13:45:07
1,33695244,use Completion Suggester match ngram query,use Completion Suggester match ngram query I...,Use Completion Suggester to match against all ...,2015-11-13 14:41:40
2,33941091,chunk document test plagiarism,chunk document test plagiarism I build plagi...,Chunking documents to test for plagiarism I a...,2015-11-26 14:28:31
3,34090734,use nltk regex pattern extract specific phrase...,use nltk regex pattern extract specific phrase...,How to use nltk regex pattern to extract a spe...,2015-12-04 14:37:13
4,34318427,itextsharp : word break split textchunk word,itextsharp : word break split textchunk word ...,itextsharp: words are broken when splitting te...,2015-12-16 17:30:43
...,...,...,...,...,...
8588,78981951,set random seed Chroma DB ?,set random seed Chroma DB ? I m experiment d...,How do I set the random seed for Chroma DB? I...,2024-09-13 11:27:10
8589,78982153,send parameter directly LLM langchain,send parameter directly LLM langchain curren...,How to send a parameter directly to LLM in lan...,2024-09-13 12:33:25
8590,78984423,Azure Document Intelligence Custom Classificat...,Azure Document Intelligence Custom Classificat...,Azure Document Intelligence Custom Classificat...,2024-09-14 06:15:24
8591,78984512,protect Routes Edge - Runtime t3 Stack / Verce...,protect Routes Edge - Runtime t3 Stack / Verce...,Protecting Routes in an Edge-Runtime with T3 S...,2024-09-14 07:17:11


In [19]:
df_to_save.to_csv('/home/kha060/PhD/Developer Challenges LLM/Stack_Overflow/CleanedLLMsPostsFromSO.csv')